In [1]:
import pandas as pd
import os
import torch
import sqlite3
from sqlite3 import Error

def show_tables(conn):
    """ show all tables in the database """
    cur = conn.cursor()
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(query)
    result = cur.fetchall()
    return [result[i][0] for i in range(len(result))]

def drop_temporary_tables(conn):
    """ drop temporary tables in the database """
    cur = conn.cursor()
    temporary_table_names = [table for table in show_tables(conn) if 'temporary' in table]
    for temporary_table_name in temporary_table_names:
        cur.execute(f"DROP TABLE IF EXISTS {temporary_table_name};")
    conn.commit()
    
def print_query(query):
    cur = conn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    for row in result:
        print(row)
        

"""
For this script, we need to have the zdb files and edf files.
"""

# get empty zdb
zdb_path = f'data/0_ponemah/Empty ZDB/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A2.20240806200742.zdb'
os.system(f'cp -f \"{zdb_path}\" \"EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A2.20240806200742.zdb\"')
zdb_path = f'EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A2.20240806200742.zdb'

try:
    conn = sqlite3.connect(zdb_path)
except Error as e:
    print(e)

# get recording start stop
cur = conn.cursor()
query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
cur.execute(query)
result = cur.fetchall()
recording_start = int(result[0][0])
query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
cur.execute(query)
result = cur.fetchall()
recording_stop = int(result[0][0])
length_ns = recording_stop - recording_start # ns
length_s = length_ns * 1e-7 # s
hh = length_s // 3600
mm = (length_s % 3600) // 60
ss = ((length_s % 3600) % 60)
print(hh,mm,ss,length_s)
print(recording_start)
print(recording_stop)

71.0 55.0 19.0 258919.0
638585572620000000
638588161810000000


In [2]:
show_tables(conn)

['internal_property',
 'logging_log_entry',
 'scoring_key',
 'scoring_marker',
 'scoring_revision',
 'scoring_revision_to_key',
 'scoring_revision_to_comment',
 'scoring_comment',
 'workspace_workspace',
 'temporary_scoring_group',
 'temporary_scoring_group_to_key',
 'temporary_scoring_group_to_comment',
 'temporary_scoring_key',
 'temporary_scoring_comment']

In [3]:
drop_temporary_tables(conn)

In [4]:
show_tables(conn)

['internal_property',
 'logging_log_entry',
 'scoring_key',
 'scoring_marker',
 'scoring_revision',
 'scoring_revision_to_key',
 'scoring_revision_to_comment',
 'scoring_comment',
 'workspace_workspace']

In [5]:
cur = conn.cursor()
cur.execute("INSERT INTO scoring_key (type) VALUES ('Automatic');")
conn.commit()
key_id = cur.lastrowid

In [6]:
print_query("SELECT * FROM scoring_key")

(1, 638725490060911738, '', '', 'Manual')
(2, 638725490218351248, '', '', 'Manual')
(3, None, None, None, 'Automatic')


In [7]:
cur = conn.cursor()
cur.execute("INSERT INTO scoring_revision (name, is_deleted, version) VALUES ('Gandalf', 0, 0);")
conn.commit()
scoring_revision_id = cur.lastrowid

In [8]:
print_query("SELECT * FROM scoring_revision")

(1, 'LSTM', 0, '', 0, '', 638725490119334504)
(2, 'LSTM', 0, '', 1, '', 638725490309286442)
(3, 'Gandalf', 0, None, 0, None, None)


In [9]:
print(scoring_revision_id,key_id)

3 3


In [10]:
cur = conn.cursor()
cur.execute(f"INSERT INTO scoring_revision_to_key (revision_id, key_id) VALUES ({scoring_revision_id}, {key_id});")
conn.commit()

In [11]:
print_query("SELECT * FROM scoring_revision_to_key")

(1, 1)
(2, 1)
(2, 2)
(3, 3)


In [12]:
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS internal_property;")
conn.commit()

In [13]:
conn.close()

In [ ]:
first_epoch_start = 638585572600000000
first_epoch_end = 638585572700000000
first_epoch_start = int(int(recording_start * 1e-8) * 1e8) # decaseconds
print(first_epoch_start)
first_epoch_start + int(1e8)